# GPU

我们回顾了过去20年计算能力的快速增长。
简而言之，自2000年以来，GPU性能每十年增长1000倍。

本节，我们将讨论如何利用这种计算性能进行研究。
首先是如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）。

我们先看看如何使用单个NVIDIA GPU进行计算。
首先，确保至少安装了一个NVIDIA GPU。
然后，下载[NVIDIA驱动和CUDA](https://developer.nvidia.com/cuda-downloads)
并按照提示设置适当的路径。
当这些准备工作完成，就可以使用`nvidia-smi`命令来(**查看显卡信息。**)

In [1]:
!nvidia-smi

Mon Sep  2 16:58:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  | 00000000:01:00.0 Off |                  N/A |
|  0%   36C    P8               5W / 165W |   5980MiB / 16380MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

在PyTorch中，每个数组都有一个设备（device），
我们通常将其称为环境（context）。
默认情况下，所有变量和相关的计算都分配给CPU。
有时环境可能是GPU。
当我们跨多个服务器部署作业时，事情会变得更加棘手。
通过智能地将数组分配给环境，
我们可以最大限度地减少在设备之间传输数据的时间。
例如，当在带有GPU的服务器上训练神经网络时，
我们通常希望模型的参数在GPU上。

要运行此部分中的程序，至少需要两个GPU。
注意，对大多数桌面计算机来说，这可能是奢侈的，但在云中很容易获得。
例如可以使用AWS EC2的多GPU实例。
本书的其他章节大都不需要多个GPU，
而本节只是为了展示数据如何在不同的设备之间传递。

## [**计算设备**]

我们可以指定用于存储和计算的设备，如CPU和GPU。
默认情况下，张量是在内存中创建的，然后使用CPU计算它。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

我们可以(**查询可用gpu的数量。**)

In [3]:
torch.cuda.device_count()

1

现在我们定义了两个方便的函数，
[**这两个函数允许我们在不存在所需所有GPU的情况下运行代码。**]

In [4]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
              for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与GPU

我们可以[**查询张量所在的设备。**]
默认情况下，张量是在CPU上创建的。

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作，
它们都必须在同一个设备上。
例如，如果我们对两个张量求和，
我们需要确保两个张量都位于同一个设备上，
否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

### [**存储在GPU上**]

有几种方法可以在GPU上存储张量。
例如，我们可以在创建张量时指定存储设备。接
下来，我们在第一个`gpu`上创建张量变量`X`。
在GPU上创建的张量只消耗这个GPU的显存。
我们可以使用`nvidia-smi`命令查看显存使用情况。
一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设我们只有一个GPU，下面的代码将在(**CPU上创建一个随机张量。**)

In [7]:
Y = torch.rand(2, 3, device=try_gpu(10))
Y

tensor([[0.6486, 0.7242, 0.7647],
        [0.6596, 0.4374, 0.2359]])

In [8]:
try:
    Y + X
except Exception as e:
    print(str(e))

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


**可以看到张量做运算必须在同一个设备上**

### 复制

如果我们[**要计算`X + Y`，我们需要决定在哪里执行这个操作**]。
例如，如下图所示，
我们可以将`X`传输到CPU并在那里执行操作。
*不要*简单地`X`加上`Y`，因为这会导致异常，
运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。
由于`Y`位于CPU上，所以我们需要将`X`移到那里，
然后才能执行相加运算。

![复制数据以在同一设备上执行操作](../assets/copyto.svg)

In [9]:
Z = X.cpu()
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [10]:
Y + Z

tensor([[1.6486, 1.7242, 1.7647],
        [1.6596, 1.4374, 1.2359]])

假设变量`Z`已经存在CPU上。
如果我们还是调用`Z.cpu()`会发生什么？
它将返回`Z`，而不会复制并分配新内存。

In [11]:
Z.cpu() is Z

True

### 旁注

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。
但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。
这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收），
然后才能继续进行更多的操作。
这就是为什么拷贝操作要格外小心。
根据经验，多个小操作比一个大操作糟糕得多。
此外，一次执行几个操作比代码中散布的许多单个操作要好得多。
如果一个设备必须等待另一个设备才能执行其他操作，
那么这样的操作可能会阻塞。
这有点像排队订购咖啡，而不像通过电话预先订购：
当客人到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时，
如果数据不在内存中，框架会首先将其复制到内存中，
这会导致额外的传输开销。
更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

## [**神经网络与GPU**]

类似地，神经网络模型可以指定设备。
下面的代码将模型参数放在GPU上。

In [12]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

在接下来的几章中，
我们将看到更多关于如何在GPU上运行模型的例子，
因为它们将变得更加计算密集。

当输入为GPU上的张量时，模型将在同一GPU上计算结果。

In [13]:
net(X)

tensor([[0.4917],
        [0.4917]], device='cuda:0', grad_fn=<AddmmBackward0>)

让我们(**确认模型参数存储在同一个GPU上。**)

In [14]:
net[0].weight.data.device

device(type='cuda', index=0)

总之，只要所有的数据和参数都在同一个设备上，
我们就可以有效地学习模型。
在下面的章节中，我们将看到几个这样的例子。

## 小结

* 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
* 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
* 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。

## 练习

1. 尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？
1. 我们应该如何在GPU上读写模型参数？
1. 测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。
1. 测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

### 练习一

1. 尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？

**解答：**

&emsp;&emsp;计算量很大的任务：使用GPU速度明显更快。

&emsp;&emsp;计算量很小的任务：CPU速度可能更快，因为数据传输到GPU需要时间。

In [15]:
import time
import torch

# 计算量较大的任务
X = torch.rand((10000, 10000))
Y = X.cuda(0)
time_start = time.time()
Z = torch.mm(X, X)
time_end = time.time()
print(f'cpu time cost: {round((time_end - time_start) * 1000, 2)}ms')
time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'gpu time cost: {round((time_end - time_start) * 1000, 2)}ms')

# 计算量很小的任务
X = torch.rand((100, 100))
Y = X.cuda(0)
time_start = time.time()
Z = torch.mm(X, X)
time_end = time.time()
print(f'cpu time cost: {round((time_end - time_start) * 1000, 2)}ms')
time_start = time.time()
Z = torch.mm(Y, Y)
time_end = time.time()
print(f'gpu time cost: {round((time_end - time_start) * 1000, 2)}ms')

cpu time cost: 3535.61ms
gpu time cost: 14.99ms
cpu time cost: 0.27ms
gpu time cost: 7.5ms


### 练习二

2. 我们应该如何在GPU上读写模型参数？

**解答**

&emsp;&emsp;使用`net.to(device)`将模型迁移到GPU上，然后再按照之前的方法读写参数。

In [16]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    """定义 MLP 类"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 定义隐藏层，输入尺寸为 20，输出尺寸为 256。
        self.output = nn.Linear(256, 10) # 定义输出层，输入尺寸为 256，输出尺寸为 10。

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

# 选择GPU，没有GPU就选CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 创建模型实例对象
net = MLP()
# 将模型参数传输到GPU上
net.to(device)
# 访问模型参数
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0624, -0.1344,  0.0151,  ...,  0.1243,  0.1772,  0.0309],
                      [-0.1788, -0.1553,  0.0989,  ...,  0.1617,  0.0595, -0.1883],
                      [ 0.0788, -0.0966,  0.2205,  ..., -0.0418, -0.1214, -0.1594],
                      ...,
                      [ 0.1584,  0.1121, -0.1025,  ..., -0.0763,  0.0542, -0.1017],
                      [ 0.1038,  0.1475,  0.1085,  ..., -0.1003, -0.1595,  0.1483],
                      [-0.0080, -0.0576, -0.0292,  ..., -0.0806,  0.1951,  0.1452]],
                     device='cuda:0')),
             ('hidden.bias',
              tensor([-1.6871e-03,  1.8661e-02,  1.7551e-01,  1.3839e-01, -2.0080e-01,
                       1.3110e-01,  1.8795e-01, -2.1238e-01, -3.4391e-02, -5.7520e-02,
                      -1.8577e-01,  1.9450e-01, -2.0548e-01,  4.5557e-02, -1.3829e-01,
                      -1.5399e-01, -5.7591e-02,  1.2907e-01,  6.7195e-03,  3.9955e-02,
                    

### 练习三

3. 测量计算1000个$100 \times 100$矩阵的矩阵乘法所需的时间，并记录输出矩阵的Frobenius范数，一次记录一个结果，而不是在GPU上保存日志并仅传输最终结果。

**解答**

&emsp;&emsp;中文版翻译有点问题，英文原版这句话是：

>Measure the time it takes to compute 1000 matrix-matrix multiplications of $100×100$ matrices and log the Frobenius norm of the output matrix one result at a time vs. keeping a log on the GPU and transferring only the final result.

&emsp;&emsp;所以这道题的本质还是希望我们做个比较。

&emsp;&emsp;实验一：仅记录1000次$100×100$矩阵相乘所用的时间，不需要打印Frobenius范数。

&emsp;&emsp;实验二：记录1000次$100×100$矩阵相乘所用的时间，并打印Frobenius范数。

In [17]:
import torch
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 生成随机矩阵
matrices = [torch.randn(100, 100).to(device) for i in range(1000)]

# 实验一：计算时间
start_time_1 = time.time()
for i in range(1000):
    result = torch.mm(matrices[i], matrices[i].t())
    frobenius_norm = torch.norm(result)

end_time_1 = time.time()
print("Time taken:", end_time_1 - start_time_1)

# 实验二：计算时间
start_time_2 = time.time()
for i in range(1000):
    result = torch.mm(matrices[i], matrices[i].t())
    frobenius_norm = torch.norm(result)
    print(frobenius_norm)

end_time_2 = time.time()
print("Time taken:", end_time_2 - start_time_2)

print(f'实验一消耗时间：{end_time_1 - start_time_1}，实验二消耗时间：{end_time_2 - start_time_2}')

Time taken: 0.04613924026489258
tensor(1447.6189, device='cuda:0')
tensor(1385.6562, device='cuda:0')
tensor(1406.6793, device='cuda:0')
tensor(1431.3994, device='cuda:0')
tensor(1366.3093, device='cuda:0')
tensor(1419.9310, device='cuda:0')
tensor(1374.8992, device='cuda:0')
tensor(1402.1981, device='cuda:0')
tensor(1396.1688, device='cuda:0')
tensor(1405.4099, device='cuda:0')
tensor(1375.2433, device='cuda:0')
tensor(1419.0358, device='cuda:0')
tensor(1429.8921, device='cuda:0')
tensor(1376.8198, device='cuda:0')
tensor(1397.0930, device='cuda:0')
tensor(1407.5486, device='cuda:0')
tensor(1426.8495, device='cuda:0')
tensor(1475.9581, device='cuda:0')
tensor(1434.7404, device='cuda:0')
tensor(1421.9675, device='cuda:0')
tensor(1382.2583, device='cuda:0')
tensor(1393.7953, device='cuda:0')
tensor(1388.4827, device='cuda:0')
tensor(1412.3043, device='cuda:0')
tensor(1403.5680, device='cuda:0')
tensor(1446.2697, device='cuda:0')
tensor(1408.5104, device='cuda:0')
tensor(1400.5610, devic

### 练习四

4. 测量同时在两个GPU上执行两个矩阵乘法与在一个GPU上按顺序执行两个矩阵乘法所需的时间。提示：应该看到近乎线性的缩放。

**解答**

&emsp;&emsp;执行两个矩阵乘法并行在两个GPU上所需的时间通常会比在单个GPU上按顺序执行这两个操作要快得多。但实际的时间取决于矩阵的大小、硬件配置和算法实现。

&emsp;&emsp;但由于笔者只有一张卡，所以只做了在单个GPU上顺序执行两个矩阵乘法的实验。

In [18]:
import torch
import time

# 创建两个随机矩阵
a = torch.randn(10000, 10000).cuda()
b = torch.randn(10000, 10000).cuda()

# 顺序执行
start_time = time.time()
c1 = torch.matmul(a, b)
c2 = torch.matmul(a, b)
end_time = time.time()
sequential_time = end_time - start_time

print(f"Sequential time: {sequential_time:.8f} seconds")

Sequential time: 0.00534868 seconds


[torch.mm和torch.matmul的区别](https://blog.csdn.net/qq_35091353/article/details/117234223)